In [5]:
import sys

sys.path.append("/home/grads/h/hasnat.md.abdullah/Snap_n_Spot/vid_llms/TimeChat")

In [9]:
#setup
!conda env create -f /home/grads/h/hasnat.md.abdullah/Snap_n_Spot/vid_llms/TimeChat/environment.yml

/home/grads/h/hasnat.md.abdullah/miniconda3/lib/python3.11/argparse.py:1978: FutureWarning: `remote_definition` is deprecated and will be removed in 25.9. Use `conda env create --file=URL` instead.
  action(self, namespace, argument_values, option_string)
Channels:
 - pytorch
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

openssl-3.0.11       | 5.2 MB    |                                       |   0% 
pip-23.2.1           | 2.6 MB    |                                       |   0% 

tornado-6.3.3        | 635 KB    |                                       |   0% 


pyzmq-25.1.0         | 459 KB    |                                       |   0% 



zeromq-4.3.4         | 331 KB    |                                       |   0% 




brotlipy-0.7.0       | 324 KB    |                                       |   0% 





urllib3-1.26.16      | 201 KB    |                                       |   0% 






traitlets-5.14.3     | 179 KB    |                            

In [6]:
print(sys.path)

['/home/grads/h/hasnat.md.abdullah/Snap_n_Spot/notebooks', '/home/grads/h/hasnat.md.abdullah/Snap_n_Spot/notebooks', '/home/grads/h/hasnat.md.abdullah/h2i_hatespeech_to_image', '/home/grads/h/hasnat.md.abdullah/h2i_hatespeech_to_image/src', '/home/grads/h/hasnat.md.abdullah/miniconda3/envs/timechat/lib/python39.zip', '/home/grads/h/hasnat.md.abdullah/miniconda3/envs/timechat/lib/python3.9', '/home/grads/h/hasnat.md.abdullah/miniconda3/envs/timechat/lib/python3.9/lib-dynload', '', '/home/grads/h/hasnat.md.abdullah/miniconda3/envs/timechat/lib/python3.9/site-packages', '/home/grads/h/hasnat.md.abdullah/Snap_n_Spot/vid_llms/TimeChat']


In [7]:
import argparse
import os
import random
import json
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from timechat.common.config import Config
from timechat.common.dist_utils import get_rank
from timechat.common.registry import registry
from timechat.conversation.conversation_video import Chat, Conversation, default_conversation,SeparatorStyle, conv_llava_llama_2
import decord
import cv2
import time
import subprocess
from decord import VideoReader
from timechat.processors.video_processor import ToTHWC, ToUint8, load_video
decord.bridge.set_bridge('torch')

# imports modules for registration
from timechat.datasets.builders import *
from timechat.models import *
from timechat.processors import *
from timechat.runners import *
from timechat.tasks import *

import random as rnd
from transformers import StoppingCriteria, StoppingCriteriaList
from PIL import Image
import gradio as gr

/home/grads/h/hasnat.md.abdullah/miniconda3/envs/timechat/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def parse_args():
    parser = argparse.ArgumentParser(description="Demo")
    parser.add_argument("--cfg-path", default='/home/grads/h/hasnat.md.abdullah/Snap_n_Spot/vid_llms/TimeChat/eval_configs/timechat.yaml', help="path to configuration file.")
    parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--num-beams", type=int, default=1)
    parser.add_argument("--temperature", type=float, default=1.0)
    parser.add_argument("--text-query", default="What is he doing?", help="question the video")
    parser.add_argument("--video-path", default='examples/hotdog.mp4', help="path to video file.")
    parser.add_argument(
        "--options",
        nargs="+",
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file (deprecate), "
        "change to --cfg-options instead.",
    )
    args = parser.parse_args(args=[])
    return args

In [17]:
!export CUDA_VISIBLE_DEVICES=0,1

In [18]:
args = parse_args()
cfg = Config(args)

DIR="/home/grads/h/hasnat.md.abdullah/Snap_n_Spot/vid_llms/TimeChat/ckpt/timechat"
MODEL_DIR=f"{DIR}/timechat_7b.pth"
model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_config.ckpt = MODEL_DIR
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))
model.eval()

vis_processor_cfg = cfg.datasets_cfg.webvid.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)

Loading VIT
Loading VIT Done
Loading Q-Former
use text input for Qformer


Using pad_token, but it is not set yet.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.83s/it]


trainable params: 0 || all params: 6,771,970,048 || trainable%: 0.0
Load first Checkpoint: /home/grads/h/hasnat.md.abdullah/Snap_n_Spot/vid_llms/TimeChat/ckpt/timechat/timechat_7b.pth


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 13.19 MiB is free. Process 32724 has 788.00 MiB memory in use. Process 22354 has 11.21 GiB memory in use. Including non-PyTorch memory, this process has 11.69 GiB memory in use. Of the allocated memory 10.68 GiB is allocated by PyTorch, and 346.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id))
print('Initialization Finished')
video, _ = load_video(
    video_path=args.video_path,
    n_frms=32,
    sampling ="uniform", return_msg = True
)
img_list = []
chat_state = conv_llava_llama_2.copy()
chat_state.system =  "You are able to understand the visual content that the user provides. Follow the instructions carefully and explain your answers in detail."
msg = chat.upload_video_without_audio(
    video_path=args.video_path, 
    conv=chat_state,
    img_list=img_list, 
    n_frms=96,
)
text_input = "You are given a cooking video from the YouCook2 dataset. Please watch the video and extract a maximum of 10 significant cooking steps. For each step, determine the starting and ending times and provide a concise description. The format should be: 'start time - end time, brief step description'. For example, ' 90 - 102 seconds, spread margarine on two slices of white bread'."
print(text_input)

chat.ask(text_input, chat_state)

num_beams = args.num_beams
temperature = args.temperature
llm_message = chat.answer(conv=chat_state,
                          img_list=img_list,
                          num_beams=num_beams,
                          temperature=temperature,
                          max_new_tokens=300,
                          max_length=2000)[0]

print(llm_message)